In [1]:
import emoji
#emoji.EMOJI_UNICODE

In [2]:
emoji_dictionary={"0":"\u2764\uFE0F",
                  "1":":baseball:",
                  "2":":grinning_face_with_big_eyes:",
                  "3":":disappointed_face:",
                  "4":":fork_and_knife:",
                  "5":":hundred_points:",
                  "6":":fire:",
                  "7":":face_blowing_a_kiss:",
                  "8":":chestnut:",
                  "9":":flexed_biceps:"}

In [3]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


## processing a custom_dataset

In [4]:
import pandas as pd
import numpy as np


In [5]:
train=pd.read_csv("./Downloads/emoji-prediction-code/train_emoji.csv",header=None)
test=pd.read_csv("./Downloads/emoji-prediction-code/test_emoji.csv",header=None)


In [6]:
Train=train.values
Test=test.values

In [7]:
X_train=Train[:,0]
Y_train=Train[:,1]
X_test=Test[:,0]
Y_test=Test[:,1]

In [8]:
for i in range (5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


## Converting Sentence into Embeddings

In [9]:
#all the sentence mapped with 50 dimensional vector using Glove 6B.50D

In [10]:
f=open("./Downloads/glove.6B.50d.txt/glove.6B.50d.txt",encoding="utf-8")


In [11]:
embeddings_index={}
cnt=0
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype="float")
    #print(word,coefs)
    embeddings_index[word]=coefs
f.close()

In [12]:
embeddings_index["talk"].shape

(50,)

## Converting sentence into vectors (output which comes from embedding_layers)

In [13]:
def embedding_output(X):
    max_Len=10
    emb_dim=50
    embedding_out=np.zeros((X.shape[0],max_Len,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        for ij in range(len(X[ix])):
            #go to every word in the current (ix) sentence
            try:
                embedding_out[ix][ij]=embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij]=np.zeros((50,))
    return embedding_out

In [14]:
embedding_matrix_train=embedding_output(X_train)
embedding_matrix_test=embedding_output(X_test)


In [15]:
print(embedding_matrix_train.shape,embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [16]:
#We converted our list into splitted form
print(X_train[1])

['I', 'am', 'proud', 'of', 'your', 'achievements']


## Create RNN/LSTM model

In [17]:
from keras.models import Sequential
from keras.layers import *

In [18]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.5))
model.add(Dense(5,activation="softmax"))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [19]:
from keras.utils import np_utils
categorical=np_utils.to_categorical
Y_train=categorical(Y_train,num_classes=5)
Y_test=categorical(Y_test,num_classes=5)

In [20]:
hist=model.fit(embedding_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 4s 1s/step - loss: 1.6076 - acc: 0.1905 - val_loss: 1.5843 - val_acc: 0.3333
Epoch 2/100
2/2 [==============================] - 0s 86ms/step - loss: 1.5955 - acc: 0.2762 - val_loss: 1.5764 - val_acc: 0.2593
Epoch 3/100
2/2 [==============================] - 0s 71ms/step - loss: 1.5440 - acc: 0.3333 - val_loss: 1.5732 - val_acc: 0.2593
Epoch 4/100
2/2 [==============================] - 0s 56ms/step - loss: 1.5451 - acc: 0.3333 - val_loss: 1.5723 - val_acc: 0.2222
Epoch 5/100
2/2 [==============================] - 0s 58ms/step - loss: 1.4956 - acc: 0.4095 - val_loss: 1.5747 - val_acc: 0.2222
Epoch 6/100
2/2 [==============================] - 0s 52ms/step - loss: 1.5030 - acc: 0.3905 - val_loss: 1.5770 - val_acc: 0.2222
Epoch 7/100
2/2 [==============================] - 0s 48ms/step - loss: 1.4703 - acc: 0.3905 - val_loss: 1.5787 - val_acc: 0.2222
Epoch 8/100
2/2 [==============================] - 0s 48ms/step - loss: 1.4223 - acc: 0.4476

2/2 [==============================] - 0s 91ms/step - loss: 0.0878 - acc: 0.9810 - val_loss: 0.9779 - val_acc: 0.6667
Epoch 64/100
2/2 [==============================] - 0s 57ms/step - loss: 0.0888 - acc: 0.9714 - val_loss: 1.0102 - val_acc: 0.7037
Epoch 65/100
2/2 [==============================] - 0s 46ms/step - loss: 0.0851 - acc: 0.9714 - val_loss: 0.8332 - val_acc: 0.8148
Epoch 66/100
2/2 [==============================] - 0s 45ms/step - loss: 0.0775 - acc: 0.9810 - val_loss: 0.9824 - val_acc: 0.6667
Epoch 67/100
2/2 [==============================] - 0s 50ms/step - loss: 0.0839 - acc: 0.9810 - val_loss: 0.9721 - val_acc: 0.7037
Epoch 68/100
2/2 [==============================] - 0s 51ms/step - loss: 0.0857 - acc: 0.9810 - val_loss: 0.9886 - val_acc: 0.7037
Epoch 69/100
2/2 [==============================] - 0s 51ms/step - loss: 0.0632 - acc: 0.9810 - val_loss: 0.9090 - val_acc: 0.7037
Epoch 70/100
2/2 [==============================] - 0s 47ms/step - loss: 0.0557 - acc: 0.9905 - 

In [21]:
##it is doing overfitting

In [22]:
predict_x=model.predict(embedding_matrix_test) 
classes_x=np.argmax(predict_x,axis=1)
classes_x

array([4, 3, 3, 2, 2, 2, 3, 2, 4, 2, 1, 2, 0, 0, 1, 3, 3, 2, 3, 4, 0, 0,
       4, 2, 3, 3, 2, 0, 3, 2, 0, 1, 3, 2, 0, 1, 2, 3, 4, 2, 2, 0, 0, 1,
       2, 2, 2, 2, 3, 1, 3, 0, 3, 2, 3, 3], dtype=int64)

In [23]:
model.evaluate(embedding_matrix_test,Y_test)

2/2 [==============================] - 0s 6ms/step - loss: 1.7696 - acc: 0.6607


[1.7696465253829956, 0.6607142686843872]

In [24]:
##Early stopping and Model_Checkpoint

In [25]:
from keras.callbacks import EarlyStopping,ModelCheckpoint


In [26]:
earlystop=EarlyStopping(monitor="val_acc",patience=10)
#checkpoint=ModelCheckpoint("best_model.h5",monitor="val_loss",verbose="True",save_best_only=True)

In [27]:
hist=model.fit(embedding_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2,
                callbacks=[earlystop])


Epoch 1/100
2/2 [==============================] - 0s 90ms/step - loss: 0.0131 - acc: 1.0000 - val_loss: 0.8123 - val_acc: 0.7778
Epoch 2/100
2/2 [==============================] - 0s 70ms/step - loss: 0.0155 - acc: 1.0000 - val_loss: 0.8271 - val_acc: 0.7407
Epoch 3/100
2/2 [==============================] - 0s 107ms/step - loss: 0.0215 - acc: 1.0000 - val_loss: 0.8617 - val_acc: 0.7407
Epoch 4/100
2/2 [==============================] - 0s 96ms/step - loss: 0.0205 - acc: 1.0000 - val_loss: 0.9025 - val_acc: 0.7037
Epoch 5/100
2/2 [==============================] - 0s 101ms/step - loss: 0.0206 - acc: 1.0000 - val_loss: 0.9055 - val_acc: 0.7407
Epoch 6/100
2/2 [==============================] - 0s 89ms/step - loss: 0.0188 - acc: 1.0000 - val_loss: 0.8855 - val_acc: 0.7037
Epoch 7/100
2/2 [==============================] - 0s 104ms/step - loss: 0.0149 - acc: 1.0000 - val_loss: 0.8542 - val_acc: 0.7778
Epoch 8/100
2/2 [==============================] - 0s 124ms/step - loss: 0.0162 - acc: 

In [28]:
model.evaluate(embedding_matrix_test,Y_test)

2/2 [==============================] - 0s 6ms/step - loss: 1.8493 - acc: 0.6429


[1.8492562770843506, 0.6428571343421936]

In [29]:
## Training_stacked LSTM model


In [30]:
stackedmodel=Sequential()
stackedmodel.add(LSTM(64,input_shape=(10,50),return_sequences=True))
stackedmodel.add(Dropout(0.5))
stackedmodel.add(LSTM(64,return_sequences=False))
stackedmodel.add(Dropout(0.5))
stackedmodel.add(Dense(5,activation="softmax"))
stackedmodel.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
stackedmodel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [31]:
hist=stackedmodel.fit(embedding_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2,
                callbacks=[earlystop])

Epoch 1/100
2/2 [==============================] - 7s 1s/step - loss: 1.6108 - acc: 0.1619 - val_loss: 1.6082 - val_acc: 0.2593
Epoch 2/100
2/2 [==============================] - 0s 60ms/step - loss: 1.5689 - acc: 0.2571 - val_loss: 1.6167 - val_acc: 0.2593
Epoch 3/100
2/2 [==============================] - 0s 62ms/step - loss: 1.5424 - acc: 0.3048 - val_loss: 1.6270 - val_acc: 0.2593
Epoch 4/100
2/2 [==============================] - 0s 61ms/step - loss: 1.5104 - acc: 0.3429 - val_loss: 1.6422 - val_acc: 0.2593
Epoch 5/100
2/2 [==============================] - 0s 61ms/step - loss: 1.4838 - acc: 0.3810 - val_loss: 1.6571 - val_acc: 0.2593
Epoch 6/100
2/2 [==============================] - 0s 63ms/step - loss: 1.4815 - acc: 0.3714 - val_loss: 1.6660 - val_acc: 0.2593
Epoch 7/100
2/2 [==============================] - 0s 62ms/step - loss: 1.4871 - acc: 0.3333 - val_loss: 1.6631 - val_acc: 0.2593
Epoch 8/100
2/2 [==============================] - 0s 63ms/step - loss: 1.4155 - acc: 0.4667

In [33]:
stackedmodel.evaluate(embedding_matrix_test,Y_test)

2/2 [==============================] - 0s 8ms/step - loss: 1.6760 - acc: 0.6429


[1.676003098487854, 0.6428571343421936]

In [35]:
predict_x=stackedmodel.predict(embedding_matrix_test) 
predict=np.argmax(predict_x,axis=1)
predict

array([4, 3, 2, 3, 2, 2, 3, 2, 4, 2, 1, 2, 0, 3, 1, 3, 3, 2, 3, 2, 3, 0,
       4, 2, 3, 3, 2, 0, 4, 2, 0, 1, 3, 2, 0, 1, 2, 3, 4, 2, 1, 0, 0, 1,
       2, 2, 2, 2, 3, 3, 3, 0, 3, 2, 3, 3], dtype=int64)

In [37]:
for i in range(30):
    print(" ".join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(predict[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😃
she got me a present
❤️
😞
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
😞
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious ha ha
😃
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😞
Congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
😃
I love taking breaks
❤️
😞
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
😃
Why are you feeling bad
😞
😞
I am upset
😞
😞
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
🍴
valentine day is near
❤️
😃
